# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,57.65,47,100,16.11,CL,1706738062
1,1,grytviken,-54.2811,-36.5092,44.04,52,91,4.23,GS,1706738062
2,2,albany,42.6001,-73.9662,36.18,86,100,1.72,US,1706737886
3,3,waitangi,-43.9535,-176.5597,64.81,98,100,4.85,NZ,1706737855
4,4,yellowknife,62.4560,-114.3525,10.92,85,75,4.61,CA,1706738062


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat', 
    geo=True,
    tiles='OSM',
    frame_width=800,
    frame_height=600,
    color= 'City',
    size= 'Humidity'
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_2 = city_data_df.loc[(city_data_df['Max Temp']< 78) & (city_data_df['Max Temp']> 60)
                           & (city_data_df['Wind Speed'] > 4.5) & (city_data_df['Cloudiness']==0)]

# Drop any rows with null values
city_data_3 =city_data_2.dropna(how='any')

# Display sample data
city_data_3

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
92,92,port elizabeth,-33.9180,25.5701,70.32,86,0,23.02,ZA,1706738074
101,101,port isabel,26.0734,-97.2086,75.29,72,0,14.97,US,1706738075
118,118,valdosta,30.8327,-83.2785,62.55,39,0,11.50,US,1706737989
150,150,margaret river,-33.9500,115.0667,64.04,67,0,6.89,AU,1706737983
205,205,tazacorte,28.6290,-17.9293,67.15,32,0,10.00,ES,1706738085
210,210,tallahassee,30.4383,-84.2807,66.04,41,0,11.50,US,1706738001
271,271,edgewater,28.9889,-80.9023,70.29,37,0,11.50,US,1706738092
300,300,geraldton,-28.7667,114.6000,72.01,49,0,5.75,AU,1706738095
304,304,harlingen,26.1906,-97.6961,77.05,35,0,10.36,US,1706738029
369,369,aghajari,30.7006,49.8315,62.19,44,0,7.83,IR,1706738103


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_2[["City", "Country", "Lat", "Lng", "Humidity"]] .copy()
hotel_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
92,port elizabeth,ZA,-33.9180,25.5701,86,
101,port isabel,US,26.0734,-97.2086,72,
118,valdosta,US,30.8327,-83.2785,39,
150,margaret river,AU,-33.9500,115.0667,67,
205,tazacorte,ES,28.6290,-17.9293,32,
210,tallahassee,US,30.4383,-84.2807,41,
271,edgewater,US,28.9889,-80.9023,37,
300,geraldton,AU,-28.7667,114.6000,49,
304,harlingen,US,26.1906,-97.6961,35,
369,aghajari,IR,30.7006,49.8315,44,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    #Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
port elizabeth - nearest hotel: Waterford Hotel
port isabel - nearest hotel: Queen Isabel Inn
valdosta - nearest hotel: Comfort Inn & Suites
margaret river - nearest hotel: Margaret River Hotel
tazacorte - nearest hotel: App Leyma
tallahassee - nearest hotel: The Governors Inn
edgewater - nearest hotel: Best Western Edgewater Inn
geraldton - nearest hotel: Ocean Centre Hotel
harlingen - nearest hotel: Holiday Inn Express
aghajari - nearest hotel: No hotel found
dakhla - nearest hotel: Hotel El Massira
remuna - nearest hotel: Hotel Shyam
atar - nearest hotel: فندق سكليل
luderitz - nearest hotel: Seaview Hotel Zum Sperrgebiet
port augusta - nearest hotel: Hotel Commonwealth
kangiwa - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
92,port elizabeth,ZA,-33.9180,25.5701,86,Waterford Hotel
101,port isabel,US,26.0734,-97.2086,72,Queen Isabel Inn
118,valdosta,US,30.8327,-83.2785,39,Comfort Inn & Suites
150,margaret river,AU,-33.9500,115.0667,67,Margaret River Hotel
205,tazacorte,ES,28.6290,-17.9293,32,App Leyma
210,tallahassee,US,30.4383,-84.2807,41,The Governors Inn
271,edgewater,US,28.9889,-80.9023,37,Best Western Edgewater Inn
300,geraldton,AU,-28.7667,114.6000,49,Ocean Centre Hotel
304,harlingen,US,26.1906,-97.6961,35,Holiday Inn Express
369,aghajari,IR,30.7006,49.8315,44,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    'Lng',
    'Lat', 
    geo=True,
    tiles='OSM',
    frame_width=800,
    frame_height=600,
    hover_cols =['Hotel Name','Country']
    
)
# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)